In [ ]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn, optim
from torch.nn.modules.loss import *
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
from torch.utils.tensorboard import SummaryWriter
import pandas as pd
from pathlib import Path
%matplotlib notebook

In [ ]:
from Loss.triplet import *
from session import *
from LR_Schedule.cyclical import Cyclical
from LR_Schedule.cos_anneal import CosAnneal
from LR_Schedule.lr_find import lr_find
from callbacks import *
from validation import *
from validation import _AccuracyMeter
import Datasets.ImageData as ImageData
from Transforms.ImageTransforms import *
import util
import Datasets.ModelData as md

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
torch.cuda.set_device(0); torch.backends.cudnn.benchmark=True;

In [ ]:
data_path = Path("/media/drake/MX500/Datasets/Kannada-MNIST")
train_path = data_path/'train.csv'
test_path = data_path/'test.csv'
df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [ ]:
labels = df[df.columns[0]].to_numpy()
inputs = df[df.columns[1:]].to_numpy().reshape(-1, 28, 28)

ids = test_df[test_df.columns[0]].to_numpy()
test_inputs = test_df[test_df.columns[1:]].to_numpy().reshape(-1, 28, 28)

inputs.shape, labels.shape

In [ ]:
class KannadaMNISTDataset(Dataset):
    def __init__(self, inputs, labels, transform):
        self.data = inputs
        self.targets = labels
        self.tsfm = transform
        
    def __len__(self): return self.targets.shape[0]
        
    def __getitem__(self, i):
        x, y = self.data[i], self.targets[i]  
        x = x.astype(np.uint8)
        x = self.tsfm(x)
        x = x.float()
        
        return x, y

In [ ]:
i_dict = md.make_partition_indices(labels.shape[0], {'train': .9, 'valid': .1})

In [ ]:
transform = transforms.Compose([transforms.ToPILImage(),
                                transforms.RandomRotation(9),
                                transforms.RandomResizedCrop(28, scale=(.95, 1.05)),
                                transforms.ToTensor(), 
                                transforms.Normalize((0.5,), (0.5,))])

train_dataset = KannadaMNISTDataset(inputs[i_dict['train']], labels[i_dict['train']], transform)
valid_dataset = KannadaMNISTDataset(inputs[i_dict['valid']], labels[i_dict['valid']], transform)
test_dataset = KannadaMNISTDataset(test_inputs, ids, transform)
trainloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valloader = DataLoader(valid_dataset, batch_size=64, shuffle=False)
testloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

In [ ]:
class TripletRegularizedCrossEntropyLoss(nn.Module):
    def __init__(self, alpha, margin):     
        super().__init__()
        self.alpha = alpha
        self.margin = margin
        
    def forward(self, x, y):
        loss = F.cross_entropy(x[-1], y)
        triplet = 0
        
        for layer in x[:-1]:
            triplet += batch_hard_triplet_loss(layer.view(layer.size(0), -1), y, self.margin)
            
        # triplet *= min(self.alpha/math.sqrt(loss.item()), 1)
        triplet *= self.alpha
            
        return loss + triplet
    
    
class CustomOneHotAccuracy(OneHotAccuracy):
    def __init__(self):
        super().__init__()
        self.reset()

    def update(self, output, label):
        super().update(output[-1], label)
        
class SelectiveSequential(nn.Module):
    def __init__(self, to_select, modules_dict):
        super(SelectiveSequential, self).__init__()
        for key, module in modules_dict.items():
            self.add_module(key, module)
        self._to_select = to_select
        self.output_layer = None
    
    def setOutputLayer(self, output_layer):
        self.output_layer = output_layer
    
    def forward(self, x):
        # list = []
        for name, module in self._modules.items():               
            x = module(x)      
            
            if name is self.output_layer:
                return x.view(x.size(0), -1)
            
            #if name in self._to_select:
            #    list.append(x)
                
        return x

In [ ]:
model = SelectiveSequential(
    ['cv1', 'cv2', 'cv3', 'cv4', 'cv5', 'cv6', 'fc1', 'fc2', 'out'],
    {'cv1': nn.Sequential(nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1), 
                          nn.BatchNorm2d(16), 
                          nn.ReLU()),
     'cv2': nn.Sequential(nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1), 
                          nn.BatchNorm2d(32), 
                          nn.ReLU()),
     'cv3': nn.Sequential(nn.MaxPool2d(kernel_size=2, stride=2),
                          nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
                          nn.BatchNorm2d(64), 
                          nn.ReLU()),
     'cv4': nn.Sequential(nn.MaxPool2d(kernel_size=2, stride=2),
                          nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=2),
                          nn.BatchNorm2d(128), 
                          nn.ReLU()),
     'cv5': nn.Sequential(nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1),
                          nn.BatchNorm2d(64), 
                          nn.ReLU()),  
     'cv6': nn.Sequential(nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1),
                          nn.BatchNorm2d(32), 
                          nn.ReLU()),
    
     'fc1': nn.Sequential(Flatten(), 
                          nn.Linear(7 * 7 * 32, 50)),
     'fc2': nn.Linear(50, 25),
     'out': nn.Linear(25, 10)})

In [ ]:
for layer, idx in zip(['cv1', 'cv2', 'cv3', 'cv4', 'cv5', 'cv6', 'fc1', 'fc2'], range(8)):
    model.setOutputLayer(layer)
    sess = Session(model, BatchHardTripletLoss(.5), optim.Adam, 1e-4)
    sess.freeze_to(idx)
    validator = Validator(valloader)
    schedule = TrainingSchedule(trainloader, [validator])
    sess.train(schedule, 5)

In [ ]:
model.setOutputLayer('out')
sess = Session(model, nn.CrossEntropyLoss(), optim.Adam, 1e-4)
sess.freeze()
lr_scheduler = CosAnneal(len(trainloader), T_mult=2)
validator = Validator(valloader, OneHotAccuracy(), save_best=True, model_dir='./triplet')
schedule = TrainingSchedule(trainloader, [lr_scheduler, validator])
sess.train(schedule, 3)

In [ ]:
sess = Session(model, nn.CrossEntropyLoss(), optim.Adam, 1e-4)
sess.unfreeze()
lr_scheduler = CosAnneal(len(trainloader), T_mult=2)
validator = Validator(valloader, OneHotAccuracy(), save_best=True, model_dir='./triplet')
schedule = TrainingSchedule(trainloader, [lr_scheduler, validator])
sess.train(schedule, 7)

In [ ]:
with EvalModel(model):
    sample = model.forward(util.to_gpu(torch.randn(64, 1, 28, 28)))
    print(sample.shape)